In [1]:
from datacube.api import API, describe_flags, make_mask, list_flag_names

import xarray as xr

dc = API()

CELL = (19, -29)

# Pixel level data filtering
This notebook steps through how to filter data using the low level Datacube bit-masking API functions.

## Load PQ Dataset (by cell)

Load the pixel quality data, for all available times, in a particular cell, into an `xarray.Dataset`.

In [2]:
pqa = dc.get_dataset_by_cell(CELL, product='pqa', platform='LANDSAT_8')
pqa

<xarray.Dataset>
Dimensions:       (time: 10, x: 4000, y: 4000)
Coordinates:
  * time          (time) datetime64[ns] 2013-03-27T23:45:53.380000 ...
  * y             (y) float64 -2.8e+06 -2.8e+06 -2.8e+06 -2.8e+06 -2.8e+06 ...
  * x             (x) float64 1.9e+06 1.9e+06 1.9e+06 1.9e+06 1.9e+06 ...
Data variables:
    crs           int32 0
    pixelquality  (time, y, x) int16 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
Attributes:
    product_version: 0.0.0
    summary: These files are experimental, short lived, and the format will change.
    title: Experimental Data files From the Australian Geoscience Data Cube - DO NOT USE
    license: Creative Commons Attribution 4.0 International (CC-BY 4.0)
    source: This data is a reprojection and retile of Landsat surface reflectance scene data.

## Describe the options available with the PQ

Print out a listing of the available bit-mask flags in this bit-masked dataset.

In [3]:
print(describe_flags(pqa))

Bits are listed from the MSB (bit 13) to the LSB (bit 0)
Bit     Value   Flag Name            Description
13      0       cloud_shadow_fmask   Cloud Shadow (Fmask)
12      0       cloud_shadow_acca    Cloud Shadow (ACCA)
11      0       cloud_fmask          Cloud (Fmask)
10      0       cloud_acca           Cloud (ACCA)
9       1       land_obs             Land observation
9       0       sea_obs              Sea observation
8       1       contiguity           All bands for this pixel contain non-null values
7       0       band_7_saturated     Band 7 is saturated
6       0       band_6_2_saturated   Band 6-2 is saturated
5       0       band_6_1_saturated   Band 6-1 is saturated
4       0       band_5_saturated     Band 5 is saturated
3       0       band_4_saturated     Band 4 is saturated
2       0       band_3_saturated     Band 3 is saturated
1       0       band_2_saturated     Band 2 is saturated
0       0       band_1_saturated     Band 1 is saturated


In [4]:
list_flag_names(pqa)

['band_1_saturated',
 'band_2_saturated',
 'band_3_saturated',
 'band_4_saturated',
 'band_5_saturated',
 'band_6_1_saturated',
 'band_6_2_saturated',
 'band_7_saturated',
 'cloud_acca',
 'cloud_fmask',
 'cloud_shadow_acca',
 'cloud_shadow_fmask',
 'contiguity',
 'land_obs',
 'sea_obs']

## Define good pixel dict

After looking at the available flags for this type of dataset, we need to define which of them we care about when masking our pixel. We define this with a dictionary of flag name to boolean value.

In [5]:
GA_GOOD_PIXEL = {name: False for name in ('band_5_saturated',
 'band_6_1_saturated',
 'cloud_shadow_acca',
 'cloud_fmask',
 'band_3_saturated',
 'band_1_saturated',
 'band_4_saturated',
 'band_2_saturated',
 'cloud_acca',
 'band_6_2_saturated',
 'cloud_shadow_fmask',
 'band_7_saturated')}
GA_GOOD_PIXEL.update(dict(contiguity=True, land_obs=True))

## Make a mask to get only good pixels

Now we have our Dataset of pixel quality data, layered by time and we have a dictionary describing which bits we care about in that Dataset. We need to combine the two to create a mask. This is done using the `make_mask` function available in the Datacube API.

In [6]:
good_pixel_mask = make_mask(pqa, **GA_GOOD_PIXEL)
good_pixel_mask

<xarray.DataArray 'pixelquality' (time: 10, y: 4000, x: 4000)>
dask.array<elemwis..., shape=(10, 4000, 4000), dtype=bool, chunksize=(1, 4000, 4000)>
Coordinates:
  * time     (time) datetime64[ns] 2013-03-27T23:45:53.380000 ...
  * y        (y) float64 -2.8e+06 -2.8e+06 -2.8e+06 -2.8e+06 -2.8e+06 ...
  * x        (x) float64 1.9e+06 1.9e+06 1.9e+06 1.9e+06 1.9e+06 1.9e+06 ...

## Load NBAR Dataset (by Cell)

Now we retrieve the NBAR data from Landsat 8 for bands 1, 2 and 3 for the same cell from which we retrieved the pixel quality data.

In [7]:
nbar = dc.get_dataset_by_cell(CELL, variables=('band_1', 'band_2', 'band_3'), product='nbar', platform='LANDSAT_8')
nbar

<xarray.Dataset>
Dimensions:  (time: 10, x: 4000, y: 4000)
Coordinates:
  * time     (time) datetime64[ns] 2013-03-27T23:45:53.380000 ...
  * y        (y) float64 -2.8e+06 -2.8e+06 -2.8e+06 -2.8e+06 -2.8e+06 ...
  * x        (x) float64 1.9e+06 1.9e+06 1.9e+06 1.9e+06 1.9e+06 1.9e+06 ...
Data variables:
    band_1   (time, y, x) int16 -999 -999 -999 -999 -999 -999 -999 -999 -999 ...
    crs      int32 0
    band_3   (time, y, x) int16 -999 -999 -999 -999 -999 -999 -999 -999 -999 ...
    band_2   (time, y, x) int16 -999 -999 -999 -999 -999 -999 -999 -999 -999 ...
Attributes:
    product_version: 0.0.0
    summary: These files are experimental, short lived, and the format will change.
    title: Experimental Data files From the Australian Geoscience Data Cube - DO NOT USE
    license: Creative Commons Attribution 4.0 International (CC-BY 4.0)
    source: This data is a reprojection and retile of Landsat surface reflectance scene data.

## Apply the mask to the surface reflection data


The loaded data can now be masked using the [`xarray.Dataset.where`](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.where.html) function, which uses the co-ordinate values in both the mask and the data to ensure that they align. It will remove any data for which the mask is not available, and will throw an error if the mask and data do not overlap.

In [8]:
nbar_good_pixels = nbar.where(good_pixel_mask)['band_1']
nbar_good_pixels

<xarray.DataArray 'band_1' (time: 10, y: 4000, x: 4000)>
dask.array<elemwis..., shape=(10, 4000, 4000), dtype=float64, chunksize=(1, 4000, 4000)>
Coordinates:
  * x        (x) float64 1.9e+06 1.9e+06 1.9e+06 1.9e+06 1.9e+06 1.9e+06 ...
  * time     (time) datetime64[ns] 2013-03-27T23:45:53.380000 ...
  * y        (y) float64 -2.8e+06 -2.8e+06 -2.8e+06 -2.8e+06 -2.8e+06 ...